In [1]:
##https://www.kaggle.com/datasets/datafiniti/electronic-products-prices
##https://www.youtube.com/watch?v=UM60uo0NUI0&list=PLdF3rLdF4ICTgEwDYcRAECQPnS8eynavM&index=3

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("Electronics.csv")
df.head(1)

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,prices.sourceURLs,asins,brand,categories,dateAdded,dateUpdated,ean,imageURLs,keys,manufacturer,manufacturerNumber,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,http://www.bestbuy.com/site/sanus-tv-wall-moun...,B00C78VIUE,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso...",2015-04-13T12:00:51Z,2018-05-12T18:59:48Z,NaN,https://images-na.ssl-images-amazon.com/images...,sanusvlf410b110inchsuperslimfullmotionmountfor...,NaN,VLF410B1,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.93796E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14592 entries, 0 to 14591
Data columns (total 31 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   14592 non-null  object 
 1   prices.amountMax     14592 non-null  float64
 2   prices.amountMin     14592 non-null  float64
 3   prices.availability  12667 non-null  object 
 4   prices.condition     13189 non-null  object 
 5   prices.currency      14592 non-null  object 
 6   prices.dateSeen      14592 non-null  object 
 7   prices.isSale        14592 non-null  bool   
 8   prices.merchant      13820 non-null  object 
 9   prices.shipping      8793 non-null   object 
 10  prices.sourceURLs    14592 non-null  object 
 11  asins                14592 non-null  object 
 12  brand                14592 non-null  object 
 13  categories           14592 non-null  object 
 14  dateAdded            14592 non-null  object 
 15  dateUpdated          14592 non-null 

In [5]:
df = df[['name','brand','categories']]

In [6]:
df.head(1)

,name,brand,categories
0,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso..."


# Cleaning Dataset:-

1) name col:-

In [7]:
df['name'][0]

"Sanus VLF410B1 10-Inch Super Slim Full-Motion Mount for 37 - 84 Inches TV's"

# Removing Non-Alpha Characters:-

punctuation,marks,symbols,whitespace,characters,and special characters such as @,#,$,%,^,&,etc.

In [8]:
df['name'] = df['name'].str.lower().replace(['^a-zA-Z0-9']," ",regex=True)

In [9]:
df['categories'] = df['categories'].str.lower().replace(['^a-zA-Z0-9']," ",regex=True)

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemer = SnowballStemmer('english')

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemer.stem(w) for w in tokens if w not in stopwords.words('english')]
    return " ".join(stemming)

In [11]:
df['name'] = df['name'].apply(lambda x: tokenization(x))
df['categories'] = df['categories'].apply(lambda x: tokenization(x))

In [12]:
df['name']

0        sanus vlf410b1 10-inch super slim full-mot mou...
1        boyton - 2500w 2.1-ch . home theater system - ...
2        boyton - 2500w 2.1-ch . home theater system - ...
3        boyton - 2500w 2.1-ch . home theater system - ...
4        boyton - 2500w 2.1-ch . home theater system - ...
                               ...                        
14587    naxa - 2.1-channel soundbar 50-watt digit ampl...
14588    naxa - 2.1-channel soundbar 50-watt digit ampl...
14589    naxa - 2.1-channel soundbar 50-watt digit ampl...
14590    naxa - 2.1-channel soundbar 50-watt digit ampl...
14591    naxa - 2.1-channel soundbar 50-watt digit ampl...
Name: name, Length: 14592, dtype: object

In [13]:
df['name_cate'] = df['name'] + " " + df['categories']

In [14]:
df['name_cate']

0        sanus vlf410b1 10-inch super slim full-mot mou...
1        boyton - 2500w 2.1-ch . home theater system - ...
2        boyton - 2500w 2.1-ch . home theater system - ...
3        boyton - 2500w 2.1-ch . home theater system - ...
4        boyton - 2500w 2.1-ch . home theater system - ...
                               ...                        
14587    naxa - 2.1-channel soundbar 50-watt digit ampl...
14588    naxa - 2.1-channel soundbar 50-watt digit ampl...
14589    naxa - 2.1-channel soundbar 50-watt digit ampl...
14590    naxa - 2.1-channel soundbar 50-watt digit ampl...
14591    naxa - 2.1-channel soundbar 50-watt digit ampl...
Name: name_cate, Length: 14592, dtype: object

# Cosine Similarity 

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
def cosine_sim(txt1,txt2):
    obj_ifid = TfidfVectorizer(tokenizer=tokenization)
    matrix = obj_ifid.fit_transform([txt1,txt2])
    similarity = cosine_similarity(matrix)[0][1]
    return similarity    

# Recommend Products

In [17]:
def recommdation(query):
    tokenization_query = tokenization(query)
    df['similarity'] = df['name_cate'].apply(lambda x:cosine_sim(tokenization_query,x))
    final_df = df.sort_values(by=['similarity'],ascending=False).head(10)
    return final_df

In [18]:
recommdation("sanus vlf410b1 10-inch super slim full-mot mount 37 - 84 inch tv")

,name,brand,categories,name_cate,similarity
0,sanus vlf410b1 10-inch super slim full-mot mou...,Sanus,"audio & video accessori , tv mount , tv access...",sanus vlf410b1 10-inch super slim full-mot mou...,0.907146
10583,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10584,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10585,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10586,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10587,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10588,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
10589,sanus - visionmount full-mot tv wall mount 42 ...,Sanus,"audio & video accessori , tv mount , tv access...",sanus - visionmount full-mot tv wall mount 42 ...,0.861258
12952,os120fm full-mot wall mount 42 70 display ( bl...,OmniMount,"audio & video accessori , tv mount , audio vis...",os120fm full-mot wall mount 42 70 display ( bl...,0.856052
12957,os120fm full-mot wall mount 42 70 display ( bl...,OmniMount,"audio & video accessori , tv mount , audio vis...",os120fm full-mot wall mount 42 70 display ( bl...,0.856052


In [19]:
df['name'][0]

"sanus vlf410b1 10-inch super slim full-mot mount 37 - 84 inch tv 's"